# Fine-tuning BERT for multi-label text classification

## 환경 설정

HuggingFace Transformers, Datasets 라이브러리를 설치합니다.

In [1]:
!pip install -q transformers datasets

     |████████████████████████████████| 4.4 MB 34.4 MB/s 
     |████████████████████████████████| 362 kB 58.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.7 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 212 kB 70.3 MB/s 
     |████████████████████████████████| 140 kB 64.8 MB/s 
     |████████████████████████████████| 1.1 MB 40.4 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 271 kB 62.1 MB/s 
     |████████████████████████████████| 144 kB 57.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Load dataset

업로드한 데이터를 로드하기 위해 드라이브에 마운트합니다.


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# GPU 확인
# !nvidia-smi    

In [4]:
# torch.cuda.is_available()

In [5]:
# 경로로 이동해 주세요.
path = '/content/drive/My Drive/projects/KlueBERT' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

In [6]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

In [7]:
# !ls

## train test로 파일을 분리합니다.

In [8]:
import pandas as pd
df = pd.read_csv("cafe_review_DB_relabeled.csv")
print(len(df))
df.head(3)

4421


,index,cafe,review,친절해요,커피가 맛있어요,인테리어가 멋져요,디저트가 맛있어요,매장이 청결해요,음료가 맛있어요,대화하기 좋아요,사진이 잘 나와요,뷰가 좋아요,가성비가 좋아요,집중하기 좋아요
0,1,더숲 초소책방,사장님이 정말 친절하세요 감동 수준 바람이 심하게 날리는 편입니다 파라솔이 무너지는...,1,0,0,1,0,0,0,0,1,0,0
1,2,더숲 초소책방,예전 초소를 멋지게 바꾸어 카페를 만든 곳이예요 워낙 유명한 곳이라 평일에도 차들과...,0,1,0,1,0,0,0,0,1,0,0
2,3,더숲 초소책방,방송을 타서 그런지 정말 사람이 많았아요 왠만하면 오후 늦게 가시길 추천 빵 맛집입...,0,0,0,1,0,0,0,0,0,0,0


In [9]:
train_df = df.sample(n=4000, random_state = 123, replace = False)
test_df = df.sample(n=len(df)-4000, random_state = 123, replace = False)

In [10]:
print(train_df.shape)
print(test_df.shape)

(4000, 14)
(421, 14)


In [11]:
train_df.to_csv("train.csv",encoding='utf-8-sig',index = False)
test_df.to_csv("test.csv",encoding='utf-8-sig',index = False)

## 미리 나눠놓은 train.csv, test.csv 파일을 불러옵니다.

In [12]:
### Q1. 데이터 파일을 지정하는 코드를 작성하시오
## 각각 train_data, test_data 의 변수에 지정하는 코드를 작성해 보시오

train_data = 'train.csv'
test_data = 'test.csv'

### datasets 를 이용해 train, test 파일을 로드합니다.

In [13]:
from datasets import load_dataset, ReadInstruction

dataset = load_dataset('csv', data_files={'train': train_data, 'test': test_data})

Using custom data configuration default-742f63eebee3e626


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-742f63eebee3e626/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

### train과 test가 로드 된것을 확인합니다.
과제를 수행할 때에는 train / validataion / test 데이터로 분리해서 사용합니다.

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'cafe', 'review', '친절해요', '커피가 맛있어요', '인테리어가 멋져요', '디저트가 맛있어요', '매장이 청결해요', '음료가 맛있어요', '대화하기 좋아요', '사진이 잘 나와요', '뷰가 좋아요', '가성비가 좋아요', '집중하기 좋아요'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['index', 'cafe', 'review', '친절해요', '커피가 맛있어요', '인테리어가 멋져요', '디저트가 맛있어요', '매장이 청결해요', '음료가 맛있어요', '대화하기 좋아요', '사진이 잘 나와요', '뷰가 좋아요', '가성비가 좋아요', '집중하기 좋아요'],
        num_rows: 421
    })
})

train 데이터의 첫 번째 데이터를 확인해봅니다.

In [15]:
example = dataset['train'][0]
example

{'cafe': '에디션덴마크 서울숲 쇼룸',
 'index': 3869,
 'review': '티 메뉴 많아요 브런치도 맛있어요',
 '가성비가 좋아요': 0,
 '대화하기 좋아요': 0,
 '디저트가 맛있어요': 1,
 '매장이 청결해요': 0,
 '뷰가 좋아요': 0,
 '사진이 잘 나와요': 0,
 '음료가 맛있어요': 0,
 '인테리어가 멋져요': 0,
 '집중하기 좋아요': 0,
 '친절해요': 0,
 '커피가 맛있어요': 0}

데이터를 살펴보면 인덱스, 카페명, 리뷰, 라벨로 이뤄져있습니다.

라벨만 따로 리스트에 저장하고 각각의 라벨을 정수에 매핑합니다. 
매핑된 것은 dictionary로 만들어줍니다.

In [16]:
labels = [label for label in dataset['train'].features.keys() if label not in ['index', 'cafe', 'review']]
id2label = {idx:label for idx, label in enumerate(labels)}

print(labels)
print(id2label)

['친절해요', '커피가 맛있어요', '인테리어가 멋져요', '디저트가 맛있어요', '매장이 청결해요', '음료가 맛있어요', '대화하기 좋아요', '사진이 잘 나와요', '뷰가 좋아요', '가성비가 좋아요', '집중하기 좋아요']
{0: '친절해요', 1: '커피가 맛있어요', 2: '인테리어가 멋져요', 3: '디저트가 맛있어요', 4: '매장이 청결해요', 5: '음료가 맛있어요', 6: '대화하기 좋아요', 7: '사진이 잘 나와요', 8: '뷰가 좋아요', 9: '가성비가 좋아요', 10: '집중하기 좋아요'}


In [17]:
### Q2. enumerate() 와 dictionary comprehension을 활용하여 label:id 의 쌍을 갖는 dictionary label2id 을 정의하시오.
## id2label 을 지정하는 방식과 유사하니 위를 참조하시면 됩니다.

label2id = {label:idx for idx, label in enumerate(labels)}
print(label2id)

{'친절해요': 0, '커피가 맛있어요': 1, '인테리어가 멋져요': 2, '디저트가 맛있어요': 3, '매장이 청결해요': 4, '음료가 맛있어요': 5, '대화하기 좋아요': 6, '사진이 잘 나와요': 7, '뷰가 좋아요': 8, '가성비가 좋아요': 9, '집중하기 좋아요': 10}


## Preprocess data

말씀드렸듯이 BERT는 텍스트가 아닌 `input_ids`를 입력으로 받습니다. 이를 위해 우선 훈련된 토크나이저를 활용해 텍스트를 토큰화합니다. 이번 예시에서는 `AutoTokenizer`를 활용해 토큰화를 진행합니다.

In [18]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base") # KlueBERT의 tokenizer를 활용합니다.

def preprocess_data(examples):
  # 배치화된 텍스트를 받습니다.
  text = examples["review"]
  # 인코딩 합니다.
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=100)
  # 라벨을 배치로 만들어줍니다.
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # numpy array로 만들기 위해 0 매트릭스를 만들어줍니다.
  labels_matrix = np.zeros((len(text), len(labels)))
  # 채웁니다.
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

인코딩!

In [19]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x7f958a889710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

결과 확인

In [20]:
example = encoded_dataset['train'][0]
print(example.keys())
print(example)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
{'input_ids': [2, 1819, 5396, 1039, 2227, 2182, 18766, 2119, 4550, 10283, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
tokenizer.decode(example['input_ids'])

'[CLS] 티 메뉴 많아요 브런치도 맛있어요 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [22]:
example['labels']

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [23]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['디저트가 맛있어요']

인코딩된 데이터를 파이토치의 포맷에 맞게 변경합니다.
[참조](https://huggingface.co/docs/datasets/v2.2.1/en/package_reference/main_classes#datasets.Dataset.set_format)

In [24]:
encoded_dataset.set_format("torch")

## 모델 정의

모델을 정의하기 위해 `transformers`에서 제공하는 `AutoModelForSequenceClassification` 모듈을 import합니다. pretrained된 모델의 weight를 로드하면 multi-label-classification을 위한 layer가 random한 weight로 생성됩니다.

In [25]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

## 모델 훈련

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 
허깅페이스에서 제공하는 Trainer API를 사용해 훈련을 시킵니다. 이를 위해 `TrainingArguments`와 `Trainer`를 정의해줘야 합니다.

* `TrainingArguments` : [문서](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments)
* `상세` : [문서](https://huggingface.co/docs/transformers/v4.20.0/en/main_classes/trainer#transformers.TrainingArguments)
* `Trainer` : [참조](https://huggingface.co/transformers/main_classes/trainer.html#id1).

In [30]:
batch_size = 8
metric_name = "f1"
epochs = 5

In [34]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"model",
    evaluation_strategy = "epoch",
    # evaluation_strategy="steps",
    # eval_steps=len(dataset['train'])/epochs,
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True
)

훈련을 위해서는 `compute_metrics` 함수를 정의해야합니다.

In [35]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # 먼저 예측값에 sigmoid를 적용합니다. shape은 다음과 같습니다. (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # 그 다음, threshold를 이용해 정수값으로 변경합니다.
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # 마지막으로 metric을 계산합니다.
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # dictionary를 리턴합니다.
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [36]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [37]:
encoded_dataset['train']['input_ids'][0]

tensor([    2,  1819,  5396,  1039,  2227,  2182, 18766,  2119,  4550, 10283,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [38]:
encoded_dataset['train']['input_ids'][0].type()

'torch.LongTensor'

In [39]:
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput([('loss',
                           tensor(0.7856, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)),
                          ('logits',
                           tensor([[ 1.1937,  0.2487, -0.4088, -0.1275, -0.7197, -0.0069, -0.5450,  0.0677,
                                     0.6232,  0.3344,  0.3629]], grad_fn=<AddmmBackward0>))])

훈련 시작!

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [41]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.235100,0.159844,0.756598,0.817209,0.555819
2,0.137000,0.095288,0.899654,0.927674,0.788599
3,0.095900,0.063817,0.937872,0.954911,0.847981
4,0.070500,0.047878,0.971477,0.979532,0.926366
5,0.057000,0.042849,0.975651,0.981565,0.938242


***** Running Evaluation *****
  Num examples = 421
  Batch size = 8
Saving model checkpoint to model/checkpoint-500
Configuration saved in model/checkpoint-500/config.json
Model weights saved in model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in model/checkpoint-500/tokenizer_config.json
Special tokens file saved in model/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 421
  Batch size = 8
Saving model checkpoint to model/checkpoint-1000
Configuration saved in model/checkpoint-1000/config.json
Model weights saved in model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in model/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 421
  Batch size = 8
Saving model checkpoint to model/checkpoint-1500
Configuration saved in model/checkpoint-1500/config.json
Model weights saved in model/checkpoint-1500/pytorch_m

TrainOutput(global_step=2500, training_loss=0.11908342742919922, metrics={'train_runtime': 618.2318, 'train_samples_per_second': 32.35, 'train_steps_per_second': 4.044, 'total_flos': 1027860612000000.0, 'train_loss': 0.11908342742919922, 'epoch': 5.0})

## Evaluate

훈련 후에는 validation data를 이용해 모델을 평가합니다.

In [45]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 421
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.9382422802850356,
 'eval_f1': 0.9756507136859781,
 'eval_loss': 0.04284878075122833,
 'eval_roc_auc': 0.9815653373507189,
 'eval_runtime': 2.7207,
 'eval_samples_per_second': 154.738,
 'eval_steps_per_second': 19.48}

## Save the best model

In [48]:
trainer.save_model("./model/checkpoint-best-model")

Saving model checkpoint to ./model/checkpoint-best-model
Configuration saved in ./model/checkpoint-best-model/config.json
Model weights saved in ./model/checkpoint-best-model/pytorch_model.bin
tokenizer config file saved in ./model/checkpoint-best-model/tokenizer_config.json
Special tokens file saved in ./model/checkpoint-best-model/special_tokens_map.json


## Inference

모델에 새로운 문장을 넣어 잘 작동하는지 확인합니다. 

In [ ]:
text = "메뉴 대부분 맛있고 사장님이 굉장히 친절하세요."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}   # input으로 들어가는 각각 tensor들이 GPU를 사용하기 위한 코드

outputs = trainer.model(**encoding)   # (**encoding) input으로 encoding된 숫자값이 들어가도록 함

모델의 결과값으로 나오는 logits의 shape은 (batch_size, num_labels) 입니다. 우리는 하나의 문장만 모델에 입력하기 때문에 batch_size가 1이 되어야 합니다.

In [ ]:
logits = outputs.logits # labels에 대해 리뷰를 기준으로 모델이 뱉은 각각의 숫자값 (실수)-> logits이라고 함
logits.shape

torch.Size([1, 11])

우리가 미리 설정해둔 라벨을 얻기 위해서는 먼저 sigmoid를 적용해야합니다. sigmoid를 적용하면 모든 점수가 0과 1사이의 실수로 변경됩니다. 이를 확률로 계산할 수 있습니다. 

그 후에는 미리 설정해둔 threshold를 넘는 점수에 대해 1로 변경해준다음 앞에서 만들었던 사전을 이용해 텍스트로 변경해줍니다.

In [ ]:
# apply sigmoid + threshold   # 확률값 중에 뭐를 기준으로 해서 결과를 뱉어 낼거냐. threshold가 높을수록 아주 높은 값만을 라벨링하게 됨
sigmoid = torch.nn.Sigmoid()   #  sigmoid 함수 호출
probs = sigmoid(logits.squeeze().cpu())  # 각각에 대해 확률값으로 바꿔주는 것 
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1  # threshold 값 : 0.1 --> probs이 0.1 이상인 값들만 1로 바꾸고, 나머지는 0으로 남김  -->  1로 해당하는 값이 최종 라벨링된 값
                                                                            # threshold 값 정하는 기준 : 경험적 or ROC 커브(threshold를 계속 바꿔가면서 그래프에 점을 찍어서 curve를 그린것, FPR가 낮고-TPR가 높은 것에 가까운 점을 갖는(AUC를 가장 높게 만드는) threshold를 찾는것)

### Q3. [optional] dictionary id2label 을 활용하여 현재 예문에 대해서 모델이 inference한 label 값을 출력해 보시오.
## predictions에서 1이 된 라벨의 id를 구한 뒤, id2label 에서 그 라벨을 구하면 됩니다.

predicted_labels = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
print(predicted_labels)

['디저트가 맛있어요']


In [ ]:
# (이론) 모델의 Loss function 설명

metric: 훈련이 잘 이루어지고 있는지 확인할 수 있는 척도
    eg. 정확도(accuracy), f1 score 등

loss function: 모델을 최적하화기 위해 정의된 함수. optimizer에 의해 최적화(감소)되는 대상(gradient의 대상)
    eg. cross-entropy, Mean-Square-Error, ... BCEWithLigitsLoss? 등
